In [32]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import spacy

In [57]:
data1 = pd.read_csv("C:\\Users\\varda\\Documents\\mltask1\\datasetmltask1.csv")
#print(data1)
data = data1.iloc[:,1].values
label = data1.iloc[:,-1].values
print(data)



['The cat is taking a nap.' 'People are enjoying the outdoors.'
 'A meal is being cooked.' 'The rain has stopped.'
 'The teacher is giving a presentation.' 'The dog is playing fetch.'
 'The sky is full of vibrant colors.'
 'An experiment is being carried out.' 'Kids are playing in the sand.'
 'The hiker successfully reached the top.'
 'Someone is creating a painting.' 'A child is laughing on the swing.'
 'Cars are driving on the road.' 'People are having a dance party.'
 'The room is filled with warmth.'
 'Plans for construction are being made.' 'The wind is gently blowing.'
 'Someone is celebrating a birthday.' 'The flowers are seeking sunlight.'
 'Someone is completing a race.' 'People are eating outdoors.'
 'Microscopic examination is taking place.'
 'Kids are enjoying the fair rides.' 'Someone is performing music.'
 'The library has organized bookshelves.' 'Bread has been baked.'
 'Someone is tending to the garden.' 'The child is amused by the toy.'
 'The couple is enjoying a roman

In [58]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Define a function for text preprocessing
def preprocess_text(text):
    doc = nlp(text)
    
    words = [token.text.lower() for token in doc if not token.is_punct and not token.is_stop]
    return words


# creating new variable to store tokenized data
tokenized_data = []

for text in data:
    # Preprocess the text
    preprocessed_words = preprocess_text(text)
    tokenized_data.append(preprocessed_words)

# You can access the preprocessed and tokenized words of a specific text, for example, the first text:
first_text_tokens = tokenized_data[0:2]
print(first_text_tokens)


[['cat', 'taking', 'nap'], ['people', 'enjoying', 'outdoors']]


In [59]:
import random

# Define a function for data augmentation
def augment_data(text, n=1):
    augmented_data = [text]
    # we can use a thesaurus or a library like NLTK or spaCy to find synonyms
    # For simplicity, i am just adding some random words as "synonyms"
    for _ in range(n):
        augmented_text = list(text)  # Copy the original text
        for i in range(len(text)):
            if random.random() < 0.9:  # Augment 10% of tokens (adjust as needed)
                # Replace with a random "synonym" (for simplicity, we use random words)
                augmented_text[i] = random.choice(["augmented", "text", "sample"])
        augmented_data.append(augmented_text)
    return augmented_data

# Define a function to generate negative samples
def generate_negative_samples(tokenized_data, n=1):
    negative_samples = []
    for text in tokenized_data:
        for _ in range(n):
            # Randomly select a different text from your dataset as a negative sample
            negative_text = random.choice(tokenized_data)
            negative_samples.append(negative_text)
    return negative_samples

# Initialize lists to store augmented and negative samples
augmented_tokenized_data = []
negative_tokenized_data = []
augmented_labels = []
negative_labels = []

# Iterate through your tokenized data and labels
for text, label in zip(tokenized_data, label):
    # Add the original text and label
    augmented_tokenized_data.append(text)
    augmented_labels.append(label)
    
    # Augment the original text (e.g., create 2 augmented versions)
    augmented_texts = augment_data(text, n=10)
    augmented_tokenized_data.extend(augmented_texts)
    augmented_labels.extend([label] * (len(augmented_texts)))
    
    # Generate negative samples (e.g., create 3 negative samples)
    negative_texts = generate_negative_samples(tokenized_data, n=10)
    negative_tokenized_data.extend(negative_texts)
    negative_labels.extend([0] * (len(negative_texts)) if label == 1 else [1] * (len(negative_texts)))

# Now, 'augmented_tokenized_data' contains augmented versions of the original text, and 'negative_tokenized_data' contains negative samples.
# 'augmented_labels' contains labels for augmented data, and 'negative_labels' contains labels for negative samples.


In [60]:
# Assuming you have already created 'augmented_tokenized_data' and 'negative_tokenized_data'

# Merge the two datasets
merged_tokenized_data = augmented_tokenized_data + negative_tokenized_data

# Assuming you also have 'augmented_labels' and 'negative_labels' for the respective datasets
# Merge the corresponding labels
merged_labels = augmented_labels + negative_labels


In [61]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Assuming you have your preprocessed and tokenized data stored in 'preprocessed_tokenized_data'
# Create a vocabulary from your tokenized data
vocabulary = set(word for words in merged_tokenized_data for word in words)

# Create a dictionary to map words to unique indices
word_to_index = {word: i for i, word in enumerate(vocabulary)}

# Create one-hot encoder
encoder = OneHotEncoder()

# Transform each text into a one-hot encoded vector
one_hot_encoded_data = []

for words in merged_tokenized_data:
    one_hot_vector = np.zeros(len(vocabulary), dtype=int)
    for word in words:
        word_index = word_to_index[word]
        one_hot_vector[word_index] = 1
    one_hot_encoded_data.append(one_hot_vector)

# one_hot_encoded_data now contains a list of one-hot encoded vectors, one for each text

# You can access the one-hot encoded vector of a specific text, for example, the first text:
first_text_one_hot = one_hot_encoded_data[0]

# You can also print the one-hot encoded vector of the first text
print(first_text_one_hot)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

# Assuming you have 'one_hot_encoded_data' as your one-hot encoded dataset
# and 'labels' as your corresponding labels (e.g., 'Satisfies' or 'Does not satisfy')

# Split the data into training and testing sets
X = one_hot_encoded_data
y = merged_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a simple multi-layer perceptron (MLP) classifier
classifier = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = classifier.predict(X_test)

# Evaluate the classifier's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


Accuracy: 99.64%
Precision: 1.00
Recall: 1.00
F1-score: 1.00
